In [8]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('./data/pre-processed-raw.csv')

In [25]:
df.head()

,credit_card,city,state,zipcode,credit_card_limit,date,transaction_dollar_amount,Long,Lat,transaction_hour,transaction_day_of_week,transaction_amount_relative_to_limit,transaction_frequency,avg_transaction_amount_24h,distance_from_common_location
0,1280981422329509,Dallas,PA,18612,6000.0,2015-08-05 00:59:19,11.94,-75.964527,41.353578,0,2,0.001990,22,75.664091,1.191098
1,1280981422329509,Dallas,PA,18612,6000.0,2015-10-29 18:23:04,5.76,-76.019703,41.311467,18,3,0.000960,316,53.062943,5.439761
2,1280981422329509,Dallas,PA,18612,6000.0,2015-10-25 17:50:48,25.84,-76.043031,41.291053,17,6,0.004307,306,53.820163,8.416689
3,1280981422329509,Dallas,PA,18612,6000.0,2015-09-05 17:39:43,68.89,-75.944299,41.327282,17,5,0.011482,152,51.474539,2.935461
4,1280981422329509,Dallas,PA,18612,6000.0,2015-09-04 14:12:59,27.01,-75.997259,41.352099,14,4,0.004502,146,52.132740,2.345063


In [26]:
# Drop unnecessary columns
df_clean = df.drop(columns=['date', 'credit_card', 'city', 'state', 'zipcode', 'credit_card_limit', 'transaction_dollar_amount', 'Long', 'Lat'])

# Check if the dataframe has any NaN or infinite values
if df_clean.isnull().values.any():
    df_clean.fillna(df_clean.mean(), inplace=True)

if np.isinf(df_clean.values).any():
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    df_clean.fillna(df_clean.mean(), inplace=True)

# Convert DataFrame to numpy array
df_array = np.array(df_clean)

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Standardize the features to have mean=0 and variance=1
scaler = StandardScaler().fit(df_array)
df_scaled = scaler.transform(df_array)

# Add a small constant to avoid division by zero during scaling
df_scaled += 1e-10

# Check for any remaining NaN values
if np.isnan(df_scaled).any():
    df_scaled = np.nan_to_num(df_scaled)

# Train the model
model = IsolationForest(contamination=0.01)
model.fit(df_scaled)

# Get anomaly scores and predictions
scores = model.decision_function(df_scaled)
anomalies = model.predict(df_scaled)

# Add anomaly scores and predictions to original dataframe
df['scores'] = scores
df['anomaly'] = anomalies

# Display rows considered as anomalies
print(df[df['anomaly']==-1])

/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


             credit_card     city state  zipcode  credit_card_limit  \
38      1280981422329509   Dallas    PA    18612             6000.0   
57      1280981422329509   Dallas    PA    18612             6000.0   
89      1280981422329509   Dallas    PA    18612             6000.0   
199     1280981422329509   Dallas    PA    18612             6000.0   
210     1280981422329509   Dallas    PA    18612             6000.0   
...                  ...      ...   ...      ...                ...   
294034  8259854075192493    Omaha    GA    31821            10000.0   
294062  8259854075192493    Omaha    GA    31821            10000.0   
294078  8259854075192493    Omaha    GA    31821            10000.0   
294137  8259854075192493    Omaha    GA    31821            10000.0   
294210  1409322756311484  Houston    PA    15342            15000.0   

                       date  transaction_dollar_amount       Long        Lat  \
38      2015-09-26 00:25:53                     860.42 -76.038250  

In [27]:
len(df), len(df[df['anomaly']==-1])

(294588, 2946)